In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np

# Stock Processing

In [16]:
def load_csv(csv_path):
    with open(csv_path, 'r', newline='') as csv_fh:
        headers = csv_fh.readline().strip().split(',')
    
    data = np.loadtxt(csv_path, delimiter = ',', skiprows = 1, dtype = "U10, float, float, float, float")
    
    return headers, data

In [53]:
h, dow = load_csv("DOW.csv")
dow = np.flip(d, axis = 0)
h, sp = load_csv("SP.csv")
dow = np.flip(d, axis = 0)
h, nas = load_csv("NASDAQ.csv")
dow = np.flip(d, axis = 0)

In [69]:
def process_stocks(data):
    monthly = []
    i = 0
    
    while (i < data.shape[0]):
        chunk = data[i:i + 30]
        month = chunk[0][0][:2]
        year = chunk[0][0][-2:]
        
        avgs = [(op + cl)/2 for (date, op, high, low, cl) in chunk if date[:2] == month]
        i += len(avgs)
        monthly.append([month + "-" + year, np.mean(avgs)])
        
    return monthly

In [81]:
with open('NASDAQ_processed.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerows(process_stocks(nas))

In [82]:
with open('NASDAQ_processed.csv', 'r', newline='') as csv_fh:
    headers = csv_fh.readline().strip().split(',')
    
    data = np.loadtxt('NASDAQ_processed.csv', delimiter = ',', dtype = "U5, float")

data = np.flip(data, axis = 0)
data[0]

('01-87', 384.22714286)

# Case Shiller

In [307]:
city = ["atlanta", "boston", "chicago", "cleveland", "dallas", "denver", "detroit", "la", "miami", \
        "minneapolis", "nyc", "phoenix", "portland", "sf", "seattle", "tampa", "dc"]

dates = ["01-91", "01-90", "01-90", "01-90", "01-00", "01-90", "01-91", "01-90", "01-90", "01-90", "01-87", \
        "01-02", "01-90", "01-90", "01-90", "01-90", "11-96"]

In [319]:
def load_case(cities, starts):
    result = []
    
    for i in range(len(cities)):
        csv_path = "Case Shiller/" + cities[i] + "_case_shiller.csv"
        data = np.loadtxt(csv_path, delimiter = ',', skiprows = 1, dtype = "U10, float")
    
        for j in range(len(data)):
            data[j][0] = data[j][0][5:7] + "-" + data[j][0][2:4]
        
        print(check_months(months, data))
        ind = 0
        while (data[ind][0] != starts[i]):
            ind += 1
        
        data = data[ind:]
        data = [[cities[i], date, val] for (date, val) in data]
        
        result += data
    
    return result

In [320]:
case_shiller_data = load_case(city, dates)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [326]:
with open('Case_Shiller_processed.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerows(case_shiller_data)

# Patents

In [303]:
def del_day(data):
    cdata = [[a, b] for (a, b) in data]
    
    for j in range(len(cdata)):
        cdata[j][0] = cdata[j][0][5:7] + "-" + cdata[j][0][2:4]
            
    return cdata

In [148]:
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [217]:
def check_months(m, arr):
    for i in range(len(arr) - 1):
        month1 = arr[i][0][:2]
        month2 = arr[i + 1][0][:2]
        
        year1 = int(arr[i][0][-2:])
        year2 = int(arr[i + 1][0][-2:])
        
        ind1 = m.index(month1)
        ind2 = m.index(month2)
        
        if ind1 != 11:
            if (ind2 - ind1 != 1):
                return False
            
        else:
            if ind2 != 0:
                return False
            if ((year2 - year1) % 100 != 1):
                return False
        
    return True

In [231]:
result = []

for sf in ["sf1", "sf2"]:
    csv_path = "Patents/" + sf + "_patents.csv"
    data = np.loadtxt(csv_path, delimiter = ',', skiprows = 1, dtype = "U10, U10")

    for i in range(len(data)):
        if data[i][1] == ".":
            data[i][1] = "0"
        
    data = [(date, int(float(pat))) for (date, pat) in data]
    
    ind = 0
    while (data[ind][0] != "1990-01-01"):
        ind += 1
        
    data = data[ind:]
    
    cdata = del_day([[x, y] for (x, y) in data])
    
    print(check_months(months, cdata))
    
    print(len(data), data[-1])
    
    result.append(data)

True
372 ('2020-12-01', 2)
True
372 ('2020-12-01', 81)


In [321]:
def load_pat(cities, starts):
    result = []
    
    for i in range(len(cities)):
        csv_path = "Patents/" + cities[i] + "_patents.csv"
        data = np.loadtxt(csv_path, delimiter = ',', skiprows = 1, dtype = "U10, U10")
        
        for k in range(len(data)):
            if data[k][1] == ".":
                data[k][1] = "0"
    
        data = [[date, int(float(pat))] for (date, pat) in data]
    
        for j in range(len(data)):
            data[j][0] = data[j][0][5:7] + "-" + data[j][0][2:4]
        
        print(check_months(months, data))
        
        ind = 0
        while (data[ind][0] != starts[i]):
            ind += 1
        
        data = data[ind:]
        data = [[cities[i], date, val] for (date, val) in data]
        
        result += data
    
    return result

In [322]:
full_patents = load_pat(city, dates)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [318]:
with open('Patents_processed.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerows(full_patents)

# Population

In [292]:
def annual_linear(m, datin):
    monthly = []
    
    for i in range(len(datin) - 1):
        year = datin[i][0][:5]
        one = datin[i][1]
        two = datin[i + 1][1]
        
        for j in range(len(m)):
            date = year + months[j] + "-01"
            pop = one + (j / len(months)) * (two - one)
            
            monthly.append((date, pop))
        
    return monthly

In [323]:
def load_pop(cities, starts):
    result = []
    
    for i in range(len(cities)):
        csv_path = "Population/" + cities[i] + "_pop.csv"
        data = np.loadtxt(csv_path, delimiter = ',', skiprows = 1, dtype = "U10, float")
    
        data = annual_linear(months, data)
    
        data = del_day(data)
        
        print(check_months(months, data))
        
        ind = 0
        while (data[ind][0] != starts[i]):
            ind += 1
        
        data = data[ind:]
        data = [[cities[i], date, val] for (date, val) in data]
        
        result += data
    
    return result

In [324]:
full_pop = load_pop(city, dates)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [325]:
with open('Population_processed.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerows(full_pop)